In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy'.


{ Session Count = 8; Grid Count = 8; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 15:12:58Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:20:52 PM	705aac94...
#1: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:16:31 PM	fe883d23...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:14:05 PM	cab2c050...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#5: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:13:56 PM	643f6a58...
#6: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#7: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 3:12:58 PM	13d0abde...


In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

705aac94-c204-413b-bf1b-8d68dd3f7310

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses= sessions.Where(s => (s.Name.Contains("iPrb0")  &&s.Name.Contains("5x16")&&s.Name.Contains("iFlx0"))); // the one plotted in the publication
ses


#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [10]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M522.3,57.1 L575.7,57.1 M45.6,224.2 L51.7,226.8 L57.9,225.3 L64.0,218.9 L70.1,204.2 L76.3,197.8
 L82.4,192.0 L88.6,152.5 L94.7,154.7 L100.8,152.4 L107.0,139.6 L113.1,139.7 L119.2,139.6 L125.4,139.6
 L131.5,139.6 L137.6,139.6 L143.8,139.6 L149.9,139.6 L156.1,139.6 L162.2,139.5 L168.3,139.6 L174.5,139.6
 L180.6,139.6 L186.7,139.6 L192.9,139.2 L199.0,139.5 L205.1,141.0 L211.3,140.4 L217.4,140.8 L223.5,141.3
 L229.7,142.6 L235.8,142.2 L242.0,141.3 L248.1,141.5 L254.2,142.3 L260.4,141.7 L266.5,141.5 L272.6,141.9
 L278.8,141.8 L284.9,141.5 L291.0,141.6 L297.2,140.9 L303.3,141.7 L309.5,143.3 L315.6,149.4 L321.7,151.1
 L327.9,175.0 L334.0,200.9 L340.1,241.8 L346.3,254.1 L352.4,269.0 L358.5,276.9 L364.7,291.6 L370.8,289.1
 L377.0,287.6 L383.1,288.8 L389.2,289.1 L395.4,288.0 L401.5,288.3 L407.6,288.5 L413.8,288.4 L419.9,288.5
 L426.0,287.9 L432.2,288.0 L438.3,288.1 L444.4,287.8 L450.6,288.0 L456.7,288.0 L462.9,288.0 L469.0,288.1
 L475.1,287.8 L481.3,288.0 L487.4,288.1 L493.5,288.0 L499.7,288.1 L505.8,288.0 L511.9,281.5 L518.1,288.0
 L524.2,288.0 L530.4,288.0 L536.5,288.0 L542.6,288.1 L548.8,288.0 L554.9,288.0 L561.0,334.2 L567.2,372.5
 L573.3,378.3 L579.4,382.0 L585.6,382.9 L591.7,383.6 L597.9,383.6 L604.0,383.6 L610.1,383.7 L616.3,383.5
 L622.4,383.5 L628.5,383.5 L634.7,383.5 L640.8,383.9 L646.9,384.0 L653.1,383.8 L659.2,383.9 L665.3,383.5
 L671.5,383.5 L677.6,383.5 L683.8,383.6 L689.9,383.6 L696.0,383.5 L702.2,383.5 L708.3,383.5 L714.4,383.6
 L720.6,383.5 L726.7,383.4 L732.8,383.5 L739.0,383.5 L745.1,383.4 L751.3,383.4 L757.4,383.4 L763.5,383.4
 L769.7,383.4 L775.8,383.4 L781.9,383.5 L788.1,383.5 L794.2,383.5 L800.3,383.4 L806.5,383.4 L812.6,383.4
 L818.8,383.4 L824.9,383.4 L831.0,383.4 L837.2,383.4 L843.3,383.5 L849.4,383.4 L855.6,383.4 L861.7,383.4
 L867.8,383.4 L874.0,383.4 L880.1,383.4 L886.2,383.4 L892.4,383.4 L898.5,383.4 L904.7,383.4 L910.8,383.4
 L916.9,383.4 L923.1,383.4 L929.2,383.4 L935.3,383.4 L941.5,383.4 L947.6,383.4 L953.7,383.4 L959.9,383.4
 L966.0,383.4 L972.2,383.4 L978.3,383.4 L984.4,383.4 L990.6,383.4 L996.7,437.6 L1002.8,448.3 L1009.0,448.0
 L1015.1,448.5 L1021.2,444.7 L1027.4,446.5 L1033.5,450.3 L1039.7,446.1 L1045.8,449.6 L1051.9,443.5 L1058.1,448.4
 L1064.2,448.7 L1070.3,448.6 L1076.5,447.5 L1082.6,447.3 L1088.7,445.7 L1094.9,445.9 L1101.0,445.6 L1107.1,445.6
 L1113.3,444.9 L1119.4,445.0 '/>

## Residual History

In [11]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M33.5,57.1 L86.9,57.1 M45.6,126.3 L49.5,128.5 L53.4,129.2 L57.3,132.0 L61.3,137.8 L65.2,140.7
 L69.1,144.1 L73.0,155.7 L76.9,170.8 L80.8,157.1 L84.7,199.7 L88.7,210.8 L92.6,216.9 L96.5,217.0
 L100.4,217.0 L104.3,217.0 L108.2,217.0 L112.1,217.0 L116.0,217.0 L120.0,217.0 L123.9,217.0 L127.8,217.0
 L131.7,217.0 L135.6,217.0 L139.5,204.2 L143.4,218.6 L147.4,216.5 L151.3,219.9 L155.2,231.4 L159.1,227.4
 L163.0,248.0 L166.9,213.8 L170.8,233.3 L174.8,239.3 L178.7,250.1 L182.6,238.8 L186.5,245.5 L190.4,244.7
 L194.3,234.5 L198.2,223.2 L202.2,221.0 L206.1,219.2 L210.0,136.8 L213.9,137.3 L217.8,139.8 L221.7,135.5
 L225.6,139.5 L229.6,160.2 L233.5,176.1 L237.4,184.5 L241.3,200.4 L245.2,236.8 L249.1,262.1 L253.0,278.7
 L256.9,289.3 L260.9,297.8 L264.8,302.2 L268.7,308.4 L272.6,309.8 L276.5,311.8 L280.4,313.4 L284.3,316.0
 L288.3,319.6 L292.2,321.6 L296.1,321.6 L300.0,321.4 L303.9,323.8 L307.8,324.1 L311.7,324.3 L315.7,324.6
 L319.6,324.4 L323.5,325.6 L327.4,325.8 L331.3,326.1 L335.2,326.2 L339.1,326.3 L343.1,326.5 L347.0,326.7
 L350.9,326.6 L354.8,326.7 L358.7,326.8 L362.6,326.7 L366.5,326.7 L370.5,326.7 L374.4,183.1 L378.3,174.0
 L382.2,267.0 L386.1,323.9 L390.0,333.2 L393.9,333.6 L397.8,334.8 L401.8,356.3 L405.7,358.4 L409.6,356.0
 L413.5,357.4 L417.4,357.7 L421.3,357.7 L425.2,363.2 L429.2,363.5 L433.1,366.6 L437.0,368.9 L440.9,379.7
 L444.8,380.3 L448.7,380.3 L452.6,384.2 L456.6,384.4 L460.5,386.9 L464.4,388.3 L468.3,388.3 L472.2,388.6
 L476.1,388.6 L480.0,391.3 L484.0,391.1 L487.9,391.5 L491.8,392.1 L495.7,392.4 L499.6,392.7 L503.5,392.8
 L507.4,392.8 L511.4,392.8 L515.3,393.2 L519.2,393.2 L523.1,393.2 L527.0,394.9 L530.9,395.4 L534.8,395.5
 L538.7,395.9 L542.7,397.3 L546.6,397.3 L550.5,398.0 L554.4,398.5 L558.3,398.5 L562.2,399.0 L566.1,399.1
 L570.1,399.4 L574.0,399.4 L577.9,399.7 L581.8,400.7 L585.7,400.8 L589.6,400.8 L593.5,401.6 L597.5,402.1
 L601.4,402.1 L605.3,402.1 L609.2,402.2 L613.1,402.4 L617.0,402.7 L620.9,403.3 L624.9,403.3 L628.8,403.3
 L632.7,403.3 L636.6,403.4 L640.5,403.4 L644.4,403.4 L648.3,403.4 L652.3,265.5 L656.2,258.6 L660.1,287.3
 L664.0,281.6 L667.9,300.3 L671.8,303.6 L675.7,301.7 L679.6,313.3 L683.6,328.3 L687.5,307.2 L691.4,327.2
 L695.3,321.9 L699.2,322.0 L703.1,359.9 L707.0,367.5 L711.0,367.5 L714.9,370.0 L718.8,373.0 L722.7,373.0
 L726.6,374.1 L730.5,373.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M33.5,81.1 L86.9,81.1 M45.6,87.0 L49.5,89.8 L53.4,90.2 L57.3,92.4 L61.3,96.2 L65.2,97.9
 L69.1,99.9 L73.0,109.1 L76.9,117.2 L80.8,110.3 L84.7,128.0 L88.7,131.6 L92.6,132.6 L96.5,132.6
 L100.4,132.7 L104.3,132.7 L108.2,132.7 L112.1,132.7 L116.0,132.7 L120.0,132.7 L123.9,132.7 L127.8,132.7
 L131.7,132.

save results to text file

In [25]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

In [22]:
si

XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [ ]:
var ses= sessions.Where(si => si.Name.Contains("iFlux0"));
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:43:47 AM	ff833be6...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:24 AM	041bb15f...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	8/25/2023 10:44:42 AM	1ae3733e...


Number of cells

In [ ]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [ ]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(ses.Count(),4);


Function used to calculate the EOCs

In [25]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

do the plot

In [32]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.WriteLine($"P={p},sess={iSess}");
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
        }
    }
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

P=0,sess=0
P=0,sess=1
P=0,sess=2
P=0,sess=3
P=1,sess=0
P=1,sess=1
P=1,sess=2
P=1,sess=3
P=2,sess=0
P=2,sess=1
P=2,sess=2
P=2,sess=3
P=3,sess=0
P=3,sess=1
P=3,sess=2
P=3,sess=3
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

print EOCs

In [33]:
EOCs

[ 0.9038108734340952, 1.921975745497684, 2.3656730498000016, 2.680293477108991 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [24]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlx0"));
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...


In [17]:
var si = sessions.Pick(0);
si

XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 3:12:58 PM	13d0abde...

plot it

In [ ]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=100;
var TStoPlot= allTs.Where(ts => ts % N == 0 || ts == (double) lts.TimeStepNumber.MajorNumber); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}